In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6925,2021-01-18T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1444,205,1649,45236,...,NaN,19.0,NaN,NaN,122007.0,0.0,1382325.0,95064.0,ITG,ITG2
6926,2021-01-18T17:00:00,ITA,9,Toscana,43.769231,11.255889,720,125,845,7488,...,NaN,9.0,NaN,NaN,128185.0,12.0,2031060.0,15337.0,ITG,ITG1
6927,2021-01-18T17:00:00,ITA,10,Umbria,43.106758,12.388247,284,49,333,4247,...,"Si fa presente come due dei ricoveri, di cui u...",4.0,NaN,NaN,32397.0,0.0,549289.0,2092.0,ITI,ITI2
6928,2021-01-18T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,45,2,47,368,...,NaN,0.0,NaN,NaN,7650.0,0.0,66538.0,152.0,ITC,ITC2


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note', 'note_test', 'note_casi', 'codice_nuts_1', 'codice_nuts_2']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6925,2021-01-18T17:00:00,19,Sicilia,1444,205,1649,45236,46885,460,1278,...,NaN,NaN,122007,1477389,911144.0,19.0,122007.0,0.0,1382325.0,95064.0
6926,2021-01-18T17:00:00,9,Toscana,720,125,845,7488,8333,129,345,...,NaN,NaN,128197,2046397,1118704.0,9.0,128185.0,12.0,2031060.0,15337.0
6927,2021-01-18T17:00:00,10,Umbria,284,49,333,4247,4580,-26,94,...,NaN,NaN,32397,551381,267595.0,4.0,32397.0,0.0,549289.0,2092.0
6928,2021-01-18T17:00:00,2,Valle d'Aosta,45,2,47,368,415,0,11,...,NaN,NaN,7650,66690,40452.0,0.0,7650.0,0.0,66538.0,152.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-01-18', '2021-01-17')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-18T17:00:00,1,Piemonte,2470,166,2636,12153,14789,-211,435,...,NaN,NaN,213195,1829660,1036864.0,7.0,212846.0,349.0,1791878.0,37782.0
1,2021-01-18T17:00:00,2,Valle d'Aosta,45,2,47,368,415,0,11,...,NaN,NaN,7650,66690,40452.0,0.0,7650.0,0.0,66538.0,152.0
2,2021-01-18T17:00:00,3,Lombardia,3553,449,4002,49562,53564,-2578,1189,...,NaN,NaN,515355,5248516,2734701.0,8.0,515233.0,122.0,5226157.0,22359.0
3,2021-01-18T17:00:00,5,Veneto,2233,336,2569,66514,69083,-1557,998,...,NaN,NaN,299170,3663538,1285644.0,10.0,299170.0,0.0,3584181.0,79357.0
4,2021-01-18T17:00:00,6,Friuli Venezia Giulia,680,63,743,11942,12685,-53,153,...,NaN,NaN,60919,1038677,371716.0,3.0,60394.0,525.0,1032465.0,6212.0
5,2021-01-18T17:00:00,7,Liguria,654,64,718,4331,5049,83,263,...,NaN,NaN,66026,777459,345256.0,4.0,66026.0,0.0,770663.0,6796.0
6,2021-01-18T17:00:00,8,Emilia-Romagna,2564,238,2802,51428,54230,-601,1153,...,NaN,NaN,203141,2820331,1301697.0,16.0,203141.0,0.0,2804121.0,16210.0
7,2021-01-18T17:00:00,9,Toscana,720,125,845,7488,8333,129,345,...,NaN,NaN,128197,2046397,1118704.0,9.0,128185.0,12.0,2031060.0,15337.0
8,2021-01-18T17:00:00,10,Umbria,284,49,333,4247,4580,-26,94,...,NaN,NaN,32397,551381,267595.0,4.0,32397.0,0.0,549289.0,2092.0
9,2021-01-18T17:00:00,11,Marche,591,85,676,9575,10251,-291,157,...,NaN,NaN,50268,621742,371928.0,2.0,50268.0,0.0,618490.0,3252.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-17T17:00:00,1,Piemonte,2460,164,2624,12376,15000,-114,495,...,NaN,NaN,212760,1818843,1034494.0,8.0,212477.0,283.0,1787844.0,30999.0
1,2021-01-17T17:00:00,2,Valle d'Aosta,41,3,44,371,415,2,18,...,NaN,NaN,7639,66544,40383.0,0.0,7639.0,0.0,66399.0,145.0
2,2021-01-17T17:00:00,3,Lombardia,3610,452,4062,52080,56142,-1856,1603,...,NaN,NaN,514166,5232178,2729492.0,22.0,514072.0,94.0,5212191.0,19987.0
3,2021-01-17T17:00:00,5,Veneto,2293,338,2631,68009,70640,-1731,1369,...,NaN,NaN,298172,3643743,1283820.0,17.0,298172.0,0.0,3575753.0,67990.0
4,2021-01-17T17:00:00,6,Friuli Venezia Giulia,677,63,740,11998,12738,-245,385,...,NaN,NaN,60766,1036684,370840.0,3.0,60274.0,492.0,1030869.0,5815.0
5,2021-01-17T17:00:00,7,Liguria,657,62,719,4247,4966,112,228,...,NaN,NaN,65763,774653,344159.0,4.0,65763.0,0.0,768944.0,5709.0
6,2021-01-17T17:00:00,8,Emilia-Romagna,2529,234,2763,52068,54831,-449,1437,...,NaN,NaN,201988,2810351,1299105.0,13.0,201988.0,0.0,2795385.0,14966.0
7,2021-01-17T17:00:00,9,Toscana,698,118,816,7388,8204,-435,406,...,NaN,NaN,127852,2039503,1116053.0,4.0,127849.0,3.0,2026003.0,13500.0
8,2021-01-17T17:00:00,10,Umbria,280,49,329,4277,4606,3,153,...,NaN,NaN,32303,550730,267377.0,1.0,32303.0,0.0,548822.0,1908.0
9,2021-01-17T17:00:00,11,Marche,590,83,673,9869,10542,-552,440,...,NaN,NaN,50111,618418,370474.0,4.0,50111.0,0.0,616150.0,2268.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-01-18T17:00:00,1,Piemonte,2470,166,2636,12153,14789,-211,435,...,NaN,NaN,213195,1829660,1036864.0,7.0,212846.0,349.0,1791878.0,37782.0
1,2021-01-18T17:00:00,2,Valle d'Aosta,45,2,47,368,415,0,11,...,NaN,NaN,7650,66690,40452.0,0.0,7650.0,0.0,66538.0,152.0
2,2021-01-18T17:00:00,3,Lombardia,3553,449,4002,49562,53564,-2578,1189,...,NaN,NaN,515355,5248516,2734701.0,8.0,515233.0,122.0,5226157.0,22359.0
3,2021-01-18T17:00:00,4,Trentino-Alto Adige,501,76,577,13899,14476,-53,221,...,0.0,0.0,58813,894340,328899.0,6.0,57774.0,1039.0,878943.0,15397.0
4,2021-01-18T17:00:00,5,Veneto,2233,336,2569,66514,69083,-1557,998,...,NaN,NaN,299170,3663538,1285644.0,10.0,299170.0,0.0,3584181.0,79357.0
5,2021-01-18T17:00:00,6,Friuli Venezia Giulia,680,63,743,11942,12685,-53,153,...,NaN,NaN,60919,1038677,371716.0,3.0,60394.0,525.0,1032465.0,6212.0
6,2021-01-18T17:00:00,7,Liguria,654,64,718,4331,5049,83,263,...,NaN,NaN,66026,777459,345256.0,4.0,66026.0,0.0,770663.0,6796.0
7,2021-01-18T17:00:00,8,Emilia-Romagna,2564,238,2802,51428,54230,-601,1153,...,NaN,NaN,203141,2820331,1301697.0,16.0,203141.0,0.0,2804121.0,16210.0
8,2021-01-18T17:00:00,9,Toscana,720,125,845,7488,8333,129,345,...,NaN,NaN,128197,2046397,1118704.0,9.0,128185.0,12.0,2031060.0,15337.0
9,2021-01-18T17:00:00,10,Umbria,284,49,333,4247,4580,-26,94,...,NaN,NaN,32397,551381,267595.0,4.0,32397.0,0.0,549289.0,2092.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,0,2021-01-17T17:00:00,1,Piemonte,2460,164,2624,12376,15000,-114,...,NaN,NaN,212760,1818843,1034494.0,8.0,212477.0,283.0,1787844.0,30999.0
1,1,2021-01-17T17:00:00,2,Valle d'Aosta,41,3,44,371,415,2,...,NaN,NaN,7639,66544,40383.0,0.0,7639.0,0.0,66399.0,145.0
2,2,2021-01-17T17:00:00,3,Lombardia,3610,452,4062,52080,56142,-1856,...,NaN,NaN,514166,5232178,2729492.0,22.0,514072.0,94.0,5212191.0,19987.0
3,19,2021-01-17T17:00:00,4,Trentino-Alto Adige,487,73,560,13969,14529,58,...,0.0,0.0,58592,891607,328294.0,7.0,57624.0,968.0,877018.0,14589.0
4,3,2021-01-17T17:00:00,5,Veneto,2293,338,2631,68009,70640,-1731,...,NaN,NaN,298172,3643743,1283820.0,17.0,298172.0,0.0,3575753.0,67990.0
5,4,2021-01-17T17:00:00,6,Friuli Venezia Giulia,677,63,740,11998,12738,-245,...,NaN,NaN,60766,1036684,370840.0,3.0,60274.0,492.0,1030869.0,5815.0
6,5,2021-01-17T17:00:00,7,Liguria,657,62,719,4247,4966,112,...,NaN,NaN,65763,774653,344159.0,4.0,65763.0,0.0,768944.0,5709.0
7,6,2021-01-17T17:00:00,8,Emilia-Romagna,2529,234,2763,52068,54831,-449,...,NaN,NaN,201988,2810351,1299105.0,13.0,201988.0,0.0,2795385.0,14966.0
8,7,2021-01-17T17:00:00,9,Toscana,698,118,816,7388,8204,-435,...,NaN,NaN,127852,2039503,1116053.0,4.0,127849.0,3.0,2026003.0,13500.0
9,8,2021-01-17T17:00:00,10,Umbria,280,49,329,4277,4606,3,...,NaN,NaN,32303,550730,267377.0,1.0,32303.0,0.0,548822.0,1908.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione
0,2021-01-18T17:00:00,1,Piemonte,2470,166,2636,12153,14789,-211,435,...,NaN,213195,1829660,1036864.0,7.0,212846.0,349.0,1791878.0,37782.0,4341375
1,2021-01-18T17:00:00,2,Valle d'Aosta,45,2,47,368,415,0,11,...,NaN,7650,66690,40452.0,0.0,7650.0,0.0,66538.0,152.0,125501
2,2021-01-18T17:00:00,3,Lombardia,3553,449,4002,49562,53564,-2578,1189,...,NaN,515355,5248516,2734701.0,8.0,515233.0,122.0,5226157.0,22359.0,10103969
3,2021-01-18T17:00:00,4,Trentino-Alto Adige,501,76,577,13899,14476,-53,221,...,0.0,58813,894340,328899.0,6.0,57774.0,1039.0,878943.0,15397.0,1074819
4,2021-01-18T17:00:00,5,Veneto,2233,336,2569,66514,69083,-1557,998,...,NaN,299170,3663538,1285644.0,10.0,299170.0,0.0,3584181.0,79357.0,4907704
5,2021-01-18T17:00:00,6,Friuli Venezia Giulia,680,63,743,11942,12685,-53,153,...,NaN,60919,1038677,371716.0,3.0,60394.0,525.0,1032465.0,6212.0,1211357
6,2021-01-18T17:00:00,7,Liguria,654,64,718,4331,5049,83,263,...,NaN,66026,777459,345256.0,4.0,66026.0,0.0,770663.0,6796.0,1543127
7,2021-01-18T17:00:00,8,Emilia-Romagna,2564,238,2802,51428,54230,-601,1153,...,NaN,203141,2820331,1301697.0,16.0,203141.0,0.0,2804121.0,16210.0,4467118
8,2021-01-18T17:00:00,9,Toscana,720,125,845,7488,8333,129,345,...,NaN,128197,2046397,1118704.0,9.0,128185.0,12.0,2031060.0,15337.0,3722729
9,2021-01-18T17:00:00,10,Umbria,284,49,333,4247,4580,-26,94,...,NaN,32397,551381,267595.0,4.0,32397.0,0.0,549289.0,2092.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione,totale_posti_terapia_intensiva
0,2021-01-18T17:00:00,1,Piemonte,2470,166,2636,12153,14789,-211,435,...,213195,1829660,1036864.0,7.0,212846.0,349.0,1791878.0,37782.0,4341375,575
1,2021-01-18T17:00:00,2,Valle d'Aosta,45,2,47,368,415,0,11,...,7650,66690,40452.0,0.0,7650.0,0.0,66538.0,152.0,125501,20
2,2021-01-18T17:00:00,3,Lombardia,3553,449,4002,49562,53564,-2578,1189,...,515355,5248516,2734701.0,8.0,515233.0,122.0,5226157.0,22359.0,10103969,1036
3,2021-01-18T17:00:00,4,Trentino-Alto Adige,501,76,577,13899,14476,-53,221,...,58813,894340,328899.0,6.0,57774.0,1039.0,878943.0,15397.0,1074819,106
4,2021-01-18T17:00:00,5,Veneto,2233,336,2569,66514,69083,-1557,998,...,299170,3663538,1285644.0,10.0,299170.0,0.0,3584181.0,79357.0,4907704,1016
5,2021-01-18T17:00:00,6,Friuli Venezia Giulia,680,63,743,11942,12685,-53,153,...,60919,1038677,371716.0,3.0,60394.0,525.0,1032465.0,6212.0,1211357,180
6,2021-01-18T17:00:00,7,Liguria,654,64,718,4331,5049,83,263,...,66026,777459,345256.0,4.0,66026.0,0.0,770663.0,6796.0,1543127,209
7,2021-01-18T17:00:00,8,Emilia-Romagna,2564,238,2802,51428,54230,-601,1153,...,203141,2820331,1301697.0,16.0,203141.0,0.0,2804121.0,16210.0,4467118,634
8,2021-01-18T17:00:00,9,Toscana,720,125,845,7488,8333,129,345,...,128197,2046397,1118704.0,9.0,128185.0,12.0,2031060.0,15337.0,3722729,523
9,2021-01-18T17:00:00,10,Umbria,284,49,333,4247,4580,-26,94,...,32397,551381,267595.0,4.0,32397.0,0.0,549289.0,2092.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-01-18T17:00:00,19,Sicilia,1444,205,1649,45236,46885,460,1278,...,34.86,441,-161,39776,5024.0,38,780,3.21,0.02572,2.46
1,2021-01-18T17:00:00,3,Lombardia,3553,449,4002,49562,53564,-2578,1189,...,43.34,-2518,-414,16338,5209.0,45,3722,7.28,0.01177,5.10
2,2021-01-18T17:00:00,8,Emilia-Romagna,2564,238,2802,51428,54230,-601,1153,...,37.54,-640,-284,9980,2592.0,51,1703,11.55,0.02581,4.55
3,2021-01-18T17:00:00,5,Veneto,2233,336,2569,66514,69083,-1557,998,...,33.07,-1495,-371,19795,1824.0,47,2508,5.04,0.02034,6.10
4,2021-01-18T17:00:00,12,Lazio,2885,315,3200,74145,77345,-165,872,...,37.19,-303,-371,15204,7139.0,16,1021,5.74,0.01487,3.25
5,2021-01-18T17:00:00,15,Campania,1448,94,1542,71120,72662,-449,714,...,18.61,-437,-307,7758,4572.0,25,1138,9.20,0.01234,3.59
6,2021-01-18T17:00:00,1,Piemonte,2470,166,2636,12153,14789,-211,435,...,28.87,-223,-60,10817,2370.0,19,627,4.02,0.01002,4.91
7,2021-01-18T17:00:00,16,Puglia,1398,143,1541,53971,55512,-608,403,...,38.75,-604,-505,3065,2059.0,26,985,13.15,0.01005,2.74
8,2021-01-18T17:00:00,9,Toscana,720,125,845,7488,8333,129,345,...,23.90,100,-61,6894,2651.0,22,194,5.00,0.00927,3.44
9,2021-01-18T17:00:00,7,Liguria,654,64,718,4331,5049,83,263,...,30.62,84,35,2806,1097.0,12,168,9.37,0.01704,4.28


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,76,30,106,71.70
1,Marche,85,58,143,59.44
2,Lombardia,449,587,1036,43.34
3,Umbria,49,68,117,41.88
4,Puglia,143,226,369,38.75
5,Emilia-Romagna,238,396,634,37.54
6,Lazio,315,532,847,37.19
7,Friuli Venezia Giulia,63,117,180,35.00
8,Sicilia,205,383,588,34.86
9,Veneto,336,680,1016,33.07


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,2021-01-16T17:00:00,22,P.A. Trento,287,46,333,2053,2386,78,186,...,NaN,NaN,24938,484054,156935.0,5.0,24682.0,256.0,481797.0,2257.0
656,2021-01-17T17:00:00,21,P.A. Bolzano,202,26,228,11871,12099,14,308,...,NaN,NaN,33519,404920,171063.0,3.0,32893.0,626.0,393203.0,11717.0
657,2021-01-17T17:00:00,22,P.A. Trento,285,47,332,2098,2430,44,135,...,NaN,NaN,25073,486687,157231.0,4.0,24731.0,342.0,483815.0,2872.0
658,2021-01-18T17:00:00,21,P.A. Bolzano,214,30,244,11869,12113,14,165,...,NaN,NaN,33684,406945,171473.0,4.0,33005.0,679.0,394618.0,12327.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,2021-01-14T17:00:00,4,Trentino-Alto Adige,528,65,593,13090,13683,20,446,...,12.0,4667.0,899.0,26.0,10.96,4.33,9.56,49.61,-2.99,0.15
326,2021-01-15T17:00:00,4,Trentino-Alto Adige,513,65,578,13537,14115,432,891,...,16.0,10666.0,993.0,447.0,11.25,4.09,8.35,89.73,0.00,3.16
327,2021-01-16T17:00:00,4,Trentino-Alto Adige,489,72,561,13910,14471,356,682,...,12.0,10931.0,884.0,373.0,12.83,3.88,6.24,77.15,10.77,2.52
328,2021-01-17T17:00:00,4,Trentino-Alto Adige,487,73,560,13969,14529,58,443,...,6.0,6825.0,1109.0,59.0,13.04,3.85,6.49,39.95,1.39,0.40


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
